<a href="https://colab.research.google.com/github/JulioBurlamaqui/MachineLearning/blob/main/CNN_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import nn, optim, no_grad, max
from torch.utils.data import DataLoader, ConcatDataset, TensorDataset, Subset
from torchvision import datasets, transforms
from time import time
import numpy as np
from sklearn.metrics import f1_score as f1
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt


In [2]:
MNIST_treino = datasets.MNIST('./', train = True, transform = transforms.ToTensor(), download=True)
MNIST_teste =  datasets.MNIST('./', train = False, transform = transforms.ToTensor(), download=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 41179849.35it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 19706473.70it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 44507815.35it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5613001.99it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [3]:
MNIST = ConcatDataset((MNIST_treino, MNIST_teste))
MNIST

In [4]:
target = [label for _, label in MNIST]

In [5]:
args = {
    'batch_size':       5,    #quantidade de amostras por iteração
    'num_threads':      4,    #número de threads do DataLoader
    #número real de batches = 20 (tamanho de cada batch * quantidade de threads)
    'num_classes':      len(MNIST_teste.classes),
    'taxa_aprendizado': 1e-3, # Encontrado euristicamente, hiperparâmetro
    'weight_decay':     5e-3, # Encontrado euristicamente, hiperparâmetro
    'num_epochs':       30,   # Encontrado euristicamente, hiperparâmetro
    'folds':            5     # Quantidade de folds da validação cruzada
}

neurons = {
    'input_size':   1 * 28 * 28, #dimensão de entrada (imagens de 28x28 bits com 1 canal de cor. precisa ser achatado para uma única dimensão)
    'hidden_size':  128, # dimensão escondida, hiperparâmetro
    'out_size':     args['num_classes']
}

In [6]:
cnn = nn.Sequential(
        ## ConvBlock 1
        nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),        # entrada: (b, 1, 32, 32) e saida: (b, 6, 28, 28)
        nn.BatchNorm2d(6),
        nn.ReLU(),
        nn.AvgPool2d(kernel_size=2, stride=2, padding=0),           # entrada: (b, 6, 28, 28) e saida: (b, 6, 14, 14)

        ## ConvBlock 2
        nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),       # entrada: (b, 6, 14, 14) e saida: (b, 16, 10, 10)
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.AvgPool2d(kernel_size=2, stride=2, padding=0),           # entrada: (b, 16, 10, 10) e saida: (b, 16, 5, 5)

        ## ConvBlock 3
        nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0),     # entrada: (b, 16, 5, 5) e saida: (b, 120, 1, 1)
        nn.BatchNorm2d(120),
        nn.ReLU(),
        nn.Flatten(),  # lineariza formando um vetor                # entrada: (b, 120, 1, 1) e saida: (b, 120*1*1) = (b, 120)

        ## DenseBlock
        nn.Linear(120, 84),                                         # entrada: (b, 120) e saida: (b, 84)
        nn.ReLU(),
        nn.Linear(84, 10),                                          # entrada: (b, 84) e saida: (b, 10)
        )

# Subindo no hardware de GPU (se disponível)
cnn = cnn.to('cuda')

In [7]:
criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = optim.Adam(cnn.parameters(), lr=args['taxa_aprendizado'], weight_decay=args['weight_decay'])

In [8]:
def train(train_loader, rede, epoch):

  # Training mode
  rede.train()

  start = time()

  epoch_loss  = []
  pred_list, rotulo_list = [], []
  for batch in train_loader:

    dado, rotulo = batch

    # Cast do dado na GPU
    dado = dado.to('cuda')
    rotulo = rotulo.to('cuda')

    # Forward
    ypred = rede(dado)
    loss = criterion(ypred, rotulo)
    epoch_loss.append(loss.cpu().data)

    _, pred = max(ypred, axis=1)
    pred_list.append(pred.cpu().numpy())
    rotulo_list.append(rotulo.cpu().numpy())

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  epoch_loss = np.asarray(epoch_loss)
  pred_list  = np.asarray(pred_list).ravel()
  rotulo_list  = np.asarray(rotulo_list).ravel()

  acc = accuracy_score(pred_list, rotulo_list)

  end = time()
  print('#################### Train ####################')
  print('Epoch %d, Loss: %.4f +/- %.4f, Acc: %.2f, Time: %.2f' % (epoch+1, epoch_loss.mean(), epoch_loss.std(), acc*100, end-start))

  return epoch_loss.mean()


In [9]:
def validate(test_loader, rede, epoch):

  # Evaluation mode
  rede.eval()

  start = time()

  epoch_loss  = []
  pred_list, rotulo_list = [], []
  with no_grad():
    for batch in test_loader:
      dado, rotulo = batch

      # Cast do dado na GPU
      dado = dado.to('cuda')
      rotulo = rotulo.to('cuda')

      # Forward
      ypred = rede(dado)
      loss = criterion(ypred, rotulo)
      epoch_loss.append(loss.cpu().data)

      _, pred = max(ypred, axis=1)
      pred_list.append(pred.cpu().numpy())
      rotulo_list.append(rotulo.cpu().numpy())

  epoch_loss = np.asarray(epoch_loss)
  pred_list  = np.asarray(pred_list).ravel()
  rotulo_list  = np.asarray(rotulo_list).ravel()

  acc = accuracy_score(pred_list, rotulo_list)

  end = time()
  print('********** Validate **********')
  print('Epoch %d, Loss: %.4f +/- %.4f, Acc: %.2f, Time: %.2f\n' % (epoch+1, epoch_loss.mean(), epoch_loss.std(), acc*100, end-start))

  return epoch_loss.mean()


In [10]:
def executa(treino, teste, rede, epocas):
  treino_loss, teste_loss = [], []
  for epoca in range(epocas):
    # Train
    treino_loss.append(train(treino, rede, epoca))

    # Validate
    teste_loss.append(validate(teste, rede, epoca))

  return treino_loss, teste_loss

In [11]:
def printa_loss(train_losses, test_losses):
  plt.figure(figsize=(20, 9))
  plt.plot(train_losses, label='Train')
  plt.plot(test_losses, label='Test', linewidth=3, alpha=0.5)
  plt.xlabel('Epochs', fontsize=16)
  plt.ylabel('Loss', fontsize=16)
  plt.title('Convergence', fontsize=16)
  plt.legend()
  plt.show()

In [12]:
kfold = StratifiedKFold(n_splits=args['folds'], shuffle=True, random_state=42)

In [13]:
def executa_fold(folds, seed, X, y, batch_size, num_threads, rede, epocas):
  if folds > 1:
    kfold = StratifiedKFold(n_splits = folds, shuffle=True, random_state=seed)

    for fold, (index_treino, index_teste) in enumerate(kfold.split(X, y)):
      print(f"Fold {fold + 1} de {folds}")

      treino_loader = DataLoader(Subset(X, index_treino),
                                 batch_size = batch_size,
                                 shuffle = True,
                                 num_workers = num_threads)

      teste_loader = DataLoader(Subset(X, index_teste),
                                batch_size = batch_size,
                                shuffle = True,
                                num_workers = num_threads)

      executa(treino_loader, teste_loader, rede, epocas)

  else:
    treino_loader = DataLoader(MNIST.datasets[0], batch_size = batch_size, shuffle = True, num_workers = num_threads)

    teste_loader = DataLoader(MNIST.datasets[1], batch_size = batch_size, shuffle = True, num_workers = num_threads)

    treino_loss, teste_loss = executa(treino_loader, teste_loader, rede, epocas)

    printa_loss(treino_loss, teste_loss)

In [ ]:
executa_fold(1, 42, MNIST[0], target, args['batch_size'], args['num_threads'], cnn, args['num_epochs'])

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#################### Train ####################
Epoch 1, Loss: 0.1643 +/- 0.2530, Acc: 95.69, Time: 63.75


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


********** Validate **********
Epoch 1, Loss: 0.0547 +/- 0.1151, Acc: 98.48, Time: 6.75



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

#################### Train ####################
Epoch 28, Loss: 0.1598 +/- 0.2420, Acc: 95.73, Time: 62.07


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


********** Validate **********
Epoch 28, Loss: 0.0663 +/- 0.1348, Acc: 98.06, Time: 5.72



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#################### Train ####################
Epoch 29, Loss: 0.1569 +/- 0.2397, Acc: 95.83, Time: 62.01


In [16]:
executa_fold(5, 42, MNIST, target, args['batch_size'], args['num_threads'], cnn, 5)

Fold 1 de 5


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#################### Train ####################
Epoch 1, Loss: 0.2607 +/- 0.3279, Acc: 92.76, Time: 58.46


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


********** Validate **********
Epoch 1, Loss: 0.0764 +/- 0.1362, Acc: 98.10, Time: 9.02



...

********** Validate **********
Epoch 4, Loss: 0.0679 +/- 0.1553, Acc: 98.26, Time: 8.80



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#################### Train ####################
Epoch 5, Loss: 0.1631 +/- 0.2460, Acc: 95.66, Time: 60.34


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


********** Validate **********
Epoch 5, Loss: 0.0595 +/- 0.1412, Acc: 98.57, Time: 8.85



# Concluimos que não havia necessidade de rodar 30 épocas, bastando apenas 5 épocas para que a perda torne-se estabilizada 🧮